In [1]:
import pandas as pd
import os
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Download dataset from "https://www.kaggle.com/datasets/nasrulhakim86/coughvid-wav"
# Extract it and add the location paths

In [3]:
audio_dataset_path = 'D:/work3/Omdena/NewDelhi_Chapter/data/archive/public_dataset/'
metadata_file_path = 'D:/work3/Omdena/NewDelhi_Chapter/data/archive/public_dataset/metadata_compiled.csv'

In [4]:
metadf = pd.read_csv(metadata_file_path)
metadf.head()

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,2020-11-25T18:58:50.488301+00:00,0.0155,7.326171,48.9,2.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,0.000000,-16.5,-71.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(metadf.columns)

Index(['uuid', 'datetime', 'cough_detected', 'SNR', 'latitude', 'longitude',
       'age', 'gender', 'respiratory_condition', 'fever_muscle_pain', 'status',
       'quality_1', 'cough_type_1', 'dyspnea_1', 'wheezing_1', 'stridor_1',
       'choking_1', 'congestion_1', 'nothing_1', 'diagnosis_1', 'severity_1',
       'quality_2', 'cough_type_2', 'dyspnea_2', 'wheezing_2', 'stridor_2',
       'choking_2', 'congestion_2', 'nothing_2', 'diagnosis_2', 'severity_2',
       'quality_3', 'cough_type_3', 'dyspnea_3', 'wheezing_3', 'stridor_3',
       'choking_3', 'congestion_3', 'nothing_3', 'diagnosis_3', 'severity_3',
       'quality_4', 'cough_type_4', 'dyspnea_4', 'wheezing_4', 'stridor_4',
       'choking_4', 'congestion_4', 'nothing_4', 'diagnosis_4', 'severity_4'],
      dtype='object')


In [6]:
metadf['status'].value_counts()

healthy        12479
symptomatic     2590
COVID-19        1155
Name: status, dtype: int64

In [310]:
#metadf['respiratory_condition'].value_counts()

False    13403
True      2821
Name: respiratory_condition, dtype: int64

In [311]:
#metadf['fever_muscle_pain'].value_counts()

False    14296
True      1928
Name: fever_muscle_pain, dtype: int64

In [312]:
#metadf['cough_detected'].value_counts()

1.0000    2816
0.0000     106
0.9956      55
0.9958      52
0.9955      51
          ... 
0.1348       1
0.0027       1
0.1947       1
0.4124       1
0.5257       1
Name: cough_detected, Length: 7212, dtype: int64

In [7]:
metadf2 = metadf[((metadf['cough_detected'] > 0.99) & (metadf['status'] == 'healthy')) |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'COVID-19')) |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'symptomatic'))]

In [369]:
# This would create a bit more balanced dataset
'''
metadf2 = metadf[((metadf['cough_detected'] > 0.996) & (metadf['status'] == 'healthy')) |
                 ((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'COVID-19')) |
                 ((metadf['cough_detected'] > 0.98) & (metadf['status'] == 'symptomatic'))]
'''

In [8]:
metadf2['status'].value_counts()

healthy        2761
symptomatic    2575
COVID-19       1133
Name: status, dtype: int64

In [389]:
#metadf2['cough_detected'].value_counts()

1.0000    2500
0.9958      42
0.9955      42
0.9936      39
0.9956      38
          ... 
0.8955       1
0.0260       1
0.5336       1
0.3786       1
0.5257       1
Name: cough_detected, Length: 1581, dtype: int64

In [316]:
#metadf2['respiratory_condition'].value_counts()

False    2878
True      982
Name: respiratory_condition, dtype: int64

In [317]:
#metadf2['fever_muscle_pain'].value_counts()

False    3095
True      765
Name: fever_muscle_pain, dtype: int64

In [319]:
#metadf2['quality_1'].value_counts()

ok      521
poor    137
good     31
Name: quality_1, dtype: int64

In [320]:
#metadf2['cough_type_1'].value_counts()

dry        368
unknown    263
wet         58
Name: cough_type_1, dtype: int64

In [321]:
#metadf2['dyspnea_1'].value_counts()

False    685
True       4
Name: dyspnea_1, dtype: int64

In [322]:
#metadf2['wheezing_1'].value_counts()

False    640
True      49
Name: wheezing_1, dtype: int64

In [323]:
#metadf2['stridor_1'].value_counts()

False    689
Name: stridor_1, dtype: int64

In [324]:
#metadf2['choking_1'].value_counts()

False    688
True       1
Name: choking_1, dtype: int64

In [325]:
#metadf2['congestion_1'].value_counts()

False    675
True      14
Name: congestion_1, dtype: int64

In [326]:
#metadf2['nothing_1'].value_counts()

True     625
False     64
Name: nothing_1, dtype: int64

In [327]:
#metadf2['diagnosis_1'].value_counts()

COVID-19               244
healthy_cough          207
lower_infection        204
upper_infection         20
obstructive_disease     14
Name: diagnosis_1, dtype: int64

In [328]:
#metadf2['severity_1'].value_counts()

mild           447
pseudocough    199
severe          31
unknown         12
Name: severity_1, dtype: int64

In [9]:
newdf = metadf2

In [272]:
'''
newdf = metadf[(((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'healthy')) 
                & (((metadf['quality_1'] == 'good') | (metadf['quality_1'] == 'ok')) |
                 ((metadf['quality_2'] == 'good') | (metadf['quality_2'] == 'ok')) |
                 ((metadf['quality_3'] == 'good') | (metadf['quality_3'] == 'ok')) |
                 ((metadf['quality_4'] == 'good') | (metadf['quality_4'] == 'ok')))) 
                 |
                 (((metadf['cough_detected'] > 0.5) & (metadf['status'] == 'symptomatic')) 
                & (((metadf['quality_1'] == 'good') | (metadf['quality_1'] == 'ok')) |
                 ((metadf['quality_2'] == 'good') | (metadf['quality_2'] == 'ok')) |
                 ((metadf['quality_3'] == 'good') | (metadf['quality_3'] == 'ok')) |
                 ((metadf['quality_4'] == 'good') | (metadf['quality_4'] == 'ok'))))
                 |
                 ((metadf['cough_detected'] > 0) & (metadf['status'] == 'COVID-19'))]
'''

In [10]:
newdf

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,NaN,NaN,21.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,NaN,female,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0029d048-898a-4c70-89c7-0815cdcf7391,2020-07-16T03:01:55.059762+00:00,1.0000,9.624196,NaN,NaN,35.0,male,True,False,...,good,dry,False,False,False,False,False,True,upper_infection,mild
13,002d28bc-7806-4dfb-9c9b-afa8cb623cac,2020-04-16T22:10:24.107938+00:00,0.9959,35.641847,NaN,NaN,46.0,female,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,00357712-dd5a-4c0a-90a4-39f1f4b9d5fd,2020-04-10T12:48:17.554497+00:00,0.0576,0.000000,46.8,6.6,NaN,male,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27534,ffd18a56-096d-40fc-9862-e5c5a8ca1fcd,2020-04-14T05:56:02.587585+00:00,1.0000,12.290065,46.5,6.6,25.0,female,False,False,...,ok,dry,False,False,False,False,False,True,COVID-19,mild
27539,ffe0658f-bade-4654-ad79-40a468aabb03,2020-04-14T01:58:32.200245+00:00,1.0000,21.960583,41.6,60.9,22.0,male,True,True,...,poor,unknown,False,False,False,False,False,False,NaN,unknown
27540,ffe13fcf-c5c2-4a6a-a9fc-e010f4f033c1,2020-04-13T21:08:50.708320+00:00,0.9485,9.966762,41.1,28.8,31.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27542,ffedc843-bfc2-4ad6-a749-2bc86bdac84a,2020-06-05T03:41:37.481463+00:00,1.0000,33.661082,-34.5,-58.5,23.0,male,False,False,...,good,dry,False,False,False,False,False,True,healthy_cough,pseudocough


In [11]:
# This indicates we have a somewhat balanced dataset
newdf['status'].value_counts()

healthy        2761
symptomatic    2575
COVID-19       1133
Name: status, dtype: int64

In [12]:
newdf = newdf.fillna("")

In [13]:
newdf.drop(newdf.loc[newdf['status']=="unknown"].index, inplace=True)
newdf.drop(newdf.loc[newdf['status']==""].index, inplace=True)
newdf.drop(newdf.loc[newdf['status']=="symptomatic"].index, inplace=True)

In [14]:
newdf

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
5,001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f,2020-04-13T22:23:06.997578+00:00,0.9968,13.146502,,,21.0,male,False,False,...,,,,,,,,,,
7,001d8e33-a4af-4edb-98ba-b03f891d9a6c,2020-05-13T01:27:42.552773+00:00,0.0306,12.713480,13.8,-89.6,,female,False,True,...,,,,,,,,,,
13,002d28bc-7806-4dfb-9c9b-afa8cb623cac,2020-04-16T22:10:24.107938+00:00,0.9959,35.641847,,,46.0,female,False,False,...,,,,,,,,,,
17,00357712-dd5a-4c0a-90a4-39f1f4b9d5fd,2020-04-10T12:48:17.554497+00:00,0.0576,0.000000,46.8,6.6,,male,True,False,...,,,,,,,,,,
27,0044cb7b-448c-44e5-8302-ad8bd106fe3e,2020-05-13T18:38:39.956383+00:00,0.9947,10.109472,41.6,-4.8,41.0,female,True,False,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27525,ffbc6bfc-bd8f-4d65-97a7-638d69f42205,2020-04-17T18:45:20.041238+00:00,0.9943,8.252409,,,35.0,male,False,False,...,,,,,,,,,,
27526,ffbca476-8b35-4797-bc8a-b7f0a2f24b55,2020-04-11T18:42:24.065752+00:00,0.7466,13.204350,41.0,28.6,,female,False,False,...,,,,,,,,,,
27534,ffd18a56-096d-40fc-9862-e5c5a8ca1fcd,2020-04-14T05:56:02.587585+00:00,1.0000,12.290065,46.5,6.6,25.0,female,False,False,...,ok,dry,False,False,False,False,False,True,COVID-19,mild
27539,ffe0658f-bade-4654-ad79-40a468aabb03,2020-04-14T01:58:32.200245+00:00,1.0000,21.960583,41.6,60.9,22.0,male,True,True,...,poor,unknown,False,False,False,False,False,False,,unknown


In [15]:
status_label_map = {"healthy":0, "COVID-19": 1}
newdf = newdf.replace({"status": status_label_map})

In [16]:
newdf.status.value_counts()

0    2761
1    1133
Name: status, dtype: int64

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name)

    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=20)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=2048, hop_length=512, n_mels=20)
    mel_scaled_features = np.mean(mel_spectrogram.T,axis=0)

    #stft = np.abs(librosa.stft(audio))
    #chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate)
    #chroma_scaled_features = np.mean(chroma.T,axis=0)


    return mfccs_scaled_features, mel_scaled_features #, chroma_scaled_features
    

In [18]:
import numpy as np
from tqdm import tqdm
# Extract Mel Frequency Cepstral Coefficients and Mel Spectogram
extracted_features=[]
for index_num,row in tqdm(newdf.iterrows()):
    file_name = os.path.join(audio_dataset_path,str(row["uuid"])+".wav")
    final_class_labels=row["status"]
    mfcc, melspec = features_extractor(file_name)
    extracted_features.append([mfcc,melspec,final_class_labels])

0it [00:00, ?it/s]

3894it [04:50, 13.42it/s]


In [19]:
### converting extracted_features to Pandas dataframe
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature1','feature2','status'])
extracted_features_df.head()

,feature1,feature2,status
0,"[-519.8622, 26.079721, -28.713432, 7.9876924, ...","[0.7552014, 1.6019976, 1.7519484, 0.89610916, ...",0
1,"[-517.534, 125.805275, -59.390606, -13.782511,...","[0.05159273, 0.09458568, 0.079764344, 0.082790...",1
2,"[-507.23135, 19.040976, -16.899456, 5.4804497,...","[1.23425, 1.3887762, 3.1613564, 5.6069775, 0.7...",0
3,"[-957.87195, -0.9061077, -0.47938585, -3.27202...","[1.8812892e-07, 4.6140246e-07, 2.4876701e-06, ...",1
4,"[-530.66626, 18.382118, -13.436043, 13.502346,...","[0.9630484, 0.69150233, 0.6723138, 0.98637956,...",0


In [20]:
extracted_features_df.shape

(3894, 3)

In [18]:
#extracted_features_df.to_parquet("mfcc120_melspec120_coughvid.parquet", index=False)

In [21]:
### Split the dataset into independent and dependent dataset
X1 = np.array(extracted_features_df['feature1'].tolist())
X2 = np.array(extracted_features_df['feature2'].tolist())
#X3 = np.array(extracted_features_df['feature3'].tolist())
X = np.concatenate((X1, X2), axis=1)
#X = np.concatenate((X, X3), axis=1)
y = np.array(extracted_features_df['status'].tolist())

In [22]:
X

array([[-5.19862183e+02,  2.60797215e+01, -2.87134323e+01, ...,
         6.74860850e-02,  6.51411572e-03,  2.23455629e-07],
       [-5.17533997e+02,  1.25805275e+02, -5.93906059e+01, ...,
         1.39014261e-07,  2.55018993e-08,  1.24391333e-08],
       [-5.07231354e+02,  1.90409756e+01, -1.68994560e+01, ...,
         1.84057932e-02,  2.10750964e-03,  6.39381099e-08],
       ...,
       [-4.52715881e+02,  6.16545258e+01, -7.46882248e+00, ...,
         6.09648637e-02,  2.67244852e-03,  1.69884615e-07],
       [-3.63734161e+02,  5.07491608e+01, -3.20017853e+01, ...,
         1.31206006e-01,  5.18728513e-03,  2.02179422e-06],
       [-4.20832428e+02,  2.41599216e+01, -7.46264648e+00, ...,
         5.03229257e-03,  4.55800124e-04,  3.49893909e-08]], dtype=float32)

In [23]:
X.shape

(3894, 40)

In [24]:
### Label Encoding
y = np.array(pd.get_dummies(y))

In [25]:
y

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [26]:
y.shape

(3894, 2)

In [27]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [28]:
from sklearn.model_selection import train_test_split
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,test_size=0.5,random_state=0)

In [29]:
X_train

array([[-5.0309824e+02,  3.2177372e+01, -9.5139322e+00, ...,
         4.2314533e-02,  9.5569873e-03,  3.6970384e-06],
       [-4.3479297e+02,  3.3146671e+01, -1.3733380e+01, ...,
         2.6959516e-02,  2.5374566e-03,  2.6322647e-08],
       [-4.9038992e+02,  3.5989269e+01, -6.6987963e+00, ...,
         1.5972402e-02,  6.0006225e-04,  5.5401738e-08],
       ...,
       [-4.3351312e+02,  8.2059555e+01, -2.7975716e+01, ...,
         3.9175409e-03,  5.4975174e-04,  2.2569317e-07],
       [-4.2224606e+02,  5.0633659e+01, -8.4757417e-01, ...,
         7.7954382e-03,  1.6793501e-04,  2.6065035e-08],
       [-3.7718286e+02,  7.0876511e+01, -1.9725002e+01, ...,
         7.2035249e-03,  3.1879969e-04,  2.1329990e-08]], dtype=float32)

In [30]:
y

array([[1, 0],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [31]:
X_train.shape

(2725, 40)

In [32]:
X_val.shape

(584, 40)

In [33]:
X_test.shape

(585, 40)

In [34]:
y_train.shape

(2725, 2)

In [35]:
y_val.shape

(584, 2)

In [36]:
y_test.shape

(585, 2)

In [ ]:
# TRAIN an XGBOOST MODEL

In [37]:
# Define the model
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()

In [38]:
# Train the model
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [39]:
# Get the predictions
y_pred = model.predict(X_test)
predictions = [np.round(x) for x in y_pred]

In [40]:
# Check the accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 78.80%


In [ ]:
# Train an ANN model

In [41]:
# Define the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

### No of classes
num_labels = y.shape[1]

model=Sequential()
#First Layer
model.add(Dense(1024,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.50))

#Second Layer
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.30))

#Third Layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Fourth Layer
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Final Layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              41984     
                                                                 
 activation (Activation)     (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 activation_1 (Activation)   (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               1

In [43]:
# Compile the model with loss and optimizer
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [44]:
# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='audio_classification_v3.0.hdf5', verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_val, y_val), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
82/86 [===========================>..] - ETA: 0s - loss: 3.4435 - accuracy: 0.5926
Epoch 1: val_loss improved from inf to 0.62360, saving model to audio_classification_v3.0.hdf5
86/86 [==============================] - 2s 14ms/step - loss: 3.3611 - accuracy: 0.5934 - val_loss: 0.6236 - val_accuracy: 0.6969
Epoch 2/200
81/86 [===========================>..] - ETA: 0s - loss: 0.8664 - accuracy: 0.6184
Epoch 2: val_loss did not improve from 0.62360
86/86 [==============================] - 1s 10ms/step - loss: 0.8619 - accuracy: 0.6158 - val_loss: 0.6353 - val_accuracy: 0.6969
Epoch 3/200
86/86 [==============================] - ETA: 0s - loss: 0.6950 - accuracy: 0.6683
Epoch 3: val_loss did not improve from 0.62360
86/86 [==============================] - 1s 13ms/step - loss: 0.6950 - accuracy: 0.6683 - val_loss: 0.6258 - val_accuracy: 0.6969
Epoch 4/200
83/86 [===========================>..] - ETA: 0s - loss: 0.6485 - accuracy: 0.6965
Epoch 4: val_loss improved from 0.62360 t

In [45]:
# Check accuracy
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7948718070983887


In [46]:
X_test[1]

array([-3.93368225e+02,  6.19166145e+01, -1.18123102e+01,  1.24733381e+01,
       -2.04239826e+01,  4.88343894e-01, -1.60470066e+01, -6.41277742e+00,
       -1.00945244e+01, -3.87368941e+00, -2.12993956e+00, -9.85071754e+00,
        1.66219723e+00, -7.17612982e+00,  1.01281452e+00, -5.07499266e+00,
       -2.87096715e+00, -2.62793350e+00, -4.56559610e+00,  2.12320614e+00,
        2.03287005e+00,  9.79424286e+00,  1.52478943e+01,  1.95722923e+01,
        7.83720732e+00,  1.25137067e+00,  9.33553636e-01,  2.18274450e+00,
        2.56465507e+00,  6.07995272e-01,  2.88458109e-01,  1.45646557e-01,
        1.17428996e-01,  1.53275549e-01,  2.80597121e-01,  8.10180977e-02,
        1.47207648e-01,  7.37456232e-02,  7.26600701e-04,  1.59219073e-08],
      dtype=float32)

In [48]:
model.predict(X_test)

19/19 [==============================] - 0s 3ms/step


array([[0.92391104, 0.07608894],
       [0.7785867 , 0.22141328],
       [0.95422995, 0.04577005],
       ...,
       [0.91480887, 0.08519111],
       [0.92381835, 0.07618158],
       [0.06672721, 0.93327284]], dtype=float32)

In [49]:
import numpy as np
from sklearn.metrics import classification_report

# Assuming you have already trained your ANN model and obtained the predictions and true labels
predictions = model.predict(X_test)  # Replace X_test with your test data
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)  # Replace y_test with your true labels

19/19 [==============================] - 0s 3ms/step


In [50]:
# Calculate precision, recall, and F1 score
report = classification_report(true_labels, predicted_labels)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.80      0.95      0.87       416
           1       0.77      0.41      0.54       169

    accuracy                           0.79       585
   macro avg       0.78      0.68      0.70       585
weighted avg       0.79      0.79      0.77       585



In [ ]:
# BUILD A CNN MODEL

In [ ]:
# Preprocess the data to match proper input shape for CNN

In [ ]:
# Define the Model

In [ ]:
# Compile the model with loss and optimizer

In [ ]:
# Train the model

In [ ]:
# Check the accuracy score

In [ ]:
# BUILD A LSTM MODEL

In [ ]:
# Preprocess the data to match proper input shape for LSTM

In [ ]:
# Define the Model

In [ ]:
# Compile the model with loss and optimizer


In [ ]:
# Train the model

In [ ]:
# Check the accuracy score
